In [1]:
#imports
import torch
import numpy as np
import matplotlib.pyplot as plt
import random
import math

In [2]:
%run dataloader.ipynb

train size:  7614
dev size:  980
test size:  994


In [3]:
batch_size=500
train_dataloader = DataLoader(train_set, collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
dev_dataloader = DataLoader(dev_set, collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_set, collate_fn=collate_fn, batch_size=batch_size, shuffle=True)

In [4]:
gpu_id = 'cuda:5'
device = torch.device(gpu_id if torch.cuda.is_available() else 'cpu')

In [16]:
# cnn
class CNN(torch.nn.Module):

    def __init__(self, hidden_layer_size_1, window_size_1, hidden_layer_size_2) -> None:
        super().__init__()
        self.window_size_1 = window_size_1
        self.w1 = torch.nn.Parameter(torch.tensor(np.random.normal(0.0, 1.0, (hidden_layer_size_1, 300, window_size_1)), dtype=torch.float64))
        self.b1 = torch.nn.Parameter(torch.zeros((hidden_layer_size_1,), dtype=torch.float64))
        self.w2 = torch.nn.Parameter(torch.tensor(np.random.normal(0.0, 1.0, (hidden_layer_size_1, hidden_layer_size_2)), dtype=torch.float64))
        self.b2 = torch.nn.Parameter(torch.zeros((hidden_layer_size_2,), dtype=torch.float64))
        self.w3 = torch.nn.Parameter(torch.tensor(np.random.normal(0.0, 1.0, (hidden_layer_size_2, 1)), dtype=torch.float64))
        self.b3 = torch.nn.Parameter(torch.zeros((1,), dtype=torch.float64))

    def forward(self, x, text_lens):
        x_t = x.transpose(1, 2)
        hidden_1_t = torch.nn.functional.leaky_relu(torch.nn.functional.conv1d(x_t, self.w1, self.b1))
        hidden = hidden_1_t.transpose(1, 2)

        pad_mask = torch.zeros(hidden.shape, dtype=torch.bool, device=device)
        for i in range(hidden.shape[0]):
            for j in range(hidden.shape[1]):
                pad_mask[i, j, :] = j < ((hidden.shape[1]) - (text_lens[i] - self.window_size_1 + 1))
        masked = torch.masked_fill(hidden, pad_mask, torch.tensor(np.inf))
        
        pooled = torch.min(masked, dim=1)[0]

        hidden_2 = torch.nn.functional.leaky_relu(pooled@self.w2 + self.b2)
        return hidden_2@self.w3 + self.b3

In [17]:
device = torch.device(gpu_id if torch.cuda.is_available() else 'cpu')
cnn = CNN(hidden_layer_size_1=512, window_size_1=2, hidden_layer_size_2=128)
cnn.to(device)

optimiser = torch.optim.Adam(cnn.parameters())
epochs = 50

print('step', 'error')
dev_accs = []
errors = []
for epoch in range(1, epochs+1):
    for batch_idx, batch in enumerate(train_dataloader, 0):
        x_p,x_c,y,hid,x_p_len=map(lambda x: x.to(device), batch)
        optimiser.zero_grad()
        output = cnn(x_p[:, :, :300], x_p_len)
        error = torch.nn.functional.binary_cross_entropy_with_logits(output, y)
        error.backward()
        optimiser.step()
    
    print(epoch, error.detach().tolist())
    errors.append(error.detach().tolist())

    with torch.no_grad():
        matches = 0
        total = 0
        for batch_idx, batch in enumerate(dev_dataloader, 0): 
            x_p,x_c,y,hid, x_p_len=map(lambda x: x.to(device), batch)
            outputs = torch.sigmoid(cnn(x_p[:, :, :300], x_p_len))
            predictions = (outputs > 0.5)*1
            matches += (y == predictions).sum()
            total += len(y)
        acc = matches / total
        dev_accs.append(float(acc))
        print('dev accuracy: {:.2%}'.format(float(acc)))

plt.plot(range(1, epochs+1), dev_accs, 'b')
plt.title('dev set accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

plt.plot(range(1, epochs+1), errors, 'r')
plt.title('train error')
plt.xlabel('epochs')
plt.ylabel('error')
plt.show()

step error


/tmp/ipykernel_2560252/1881725630.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  context_w=np.array(self.language_sdk[hid]['context_embedding_indexes'])
/tmp/ipykernel_2560252/1881725630.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  context_of=np.array(self.word_aligned_openface_sdk[hid]['context_features'])
/tmp/ipykernel_2560252/1881725630.py:103: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you 

1 55.85762057616649
dev accuracy: 47.47%
2 46.6366300580128
dev accuracy: 50.15%
3 39.52416937393684
dev accuracy: 49.02%


KeyboardInterrupt: 

In [8]:
with torch.no_grad():
    matches = 0
    total = 0
    for batch_idx, batch in enumerate(test_dataloader, 0): 
        x_p,x_c,y,hid, x_p_len=map(lambda x: x.to(device), batch)
        outputs = torch.sigmoid(cnn(x_p[:, :, :300], x_p_len))
        predictions = (outputs > 0.5)*1
        matches += (y == predictions).sum()
        total += len(y)
    acc = matches / total
    print('test accuracy: {:.2%}'.format(float(acc)))

/tmp/ipykernel_1187940/1881725630.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  context_w=np.array(self.language_sdk[hid]['context_embedding_indexes'])
/tmp/ipykernel_1187940/1881725630.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  context_of=np.array(self.word_aligned_openface_sdk[hid]['context_features'])
/tmp/ipykernel_1187940/1881725630.py:103: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you 

test accuracy: 53.47%
